In [1]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext

#set some display settings for notebooks
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#add headers in case you use chromedriver (captchas are no fun); namely used for chromedriver
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [2]:
#create url variables for each zillow page
with requests.Session() as s:
    city = 'denver/' #*****change this city to what you want!!!!*****
    
    url = 'https://www.zillow.com/homes/for_sale/'+city
    url2 = 'https://www.zillow.com/homes/for_sale/'+city+'/2_p/'
    url3 = 'https://www.zillow.com/homes/for_sale/'+city+'/3_p/'
    url4 = 'https://www.zillow.com/homes/for_sale/'+city+'/4_p/'
    url5 = 'https://www.zillow.com/homes/for_sale/'+city+'/5_p/'
    url6 = 'https://www.zillow.com/homes/for_sale/'+city+'/6_p/'
    url7 = 'https://www.zillow.com/homes/for_sale/'+city+'/7_p/'
    url8 = 'https://www.zillow.com/homes/for_sale/'+city+'/8_p/'
    url9 = 'https://www.zillow.com/homes/for_sale/'+city+'/9_p/'
    url10 = 'https://www.zillow.com/homes/for_sale/'+city+'/10_p/'

    r = s.get(url, headers=req_headers)
    r2 = s.get(url2, headers=req_headers)
    r3 = s.get(url3, headers=req_headers)
    r4 = s.get(url4, headers=req_headers)
    r5 = s.get(url5, headers=req_headers)
    r6 = s.get(url6, headers=req_headers)
    r7 = s.get(url7, headers=req_headers)
    r8 = s.get(url8, headers=req_headers)
    r9 = s.get(url9, headers=req_headers)
    r10 = s.get(url10, headers=req_headers)
    
    url_links = [url, url2, url3, url4, url5, url6, url7, url8, url9, url10]

#add contents of urls to soup variable from each url
soup = BeautifulSoup(r.content, 'html.parser')
soup1 = BeautifulSoup(r2.content, 'html.parser')
soup2 = BeautifulSoup(r3.content, 'html.parser')
soup3 = BeautifulSoup(r4.content, 'html.parser')
soup4 = BeautifulSoup(r5.content, 'html.parser')
soup5 = BeautifulSoup(r6.content, 'html.parser')
soup6 = BeautifulSoup(r7.content, 'html.parser')
soup7 = BeautifulSoup(r8.content, 'html.parser')
soup8 = BeautifulSoup(r9.content, 'html.parser')
soup9 = BeautifulSoup(r10.content, 'html.parser')

page_links = [soup, soup1, soup2, soup3, soup4, soup5, soup6, soup7, soup8, soup9]

#create the first two dataframes
df_master = pd.DataFrame()
df = pd.DataFrame()
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()
df10 = pd.DataFrame()

dataframe_list = [df,df1,df2,df3, df4,df5,df6,df7,df8,df9,df10]

In [3]:
for df, soup in zip(dataframe_list, page_links):
    #all for loops are pulling the specified variable using beautiful soup and inserting into said variable
    for i in soup:
        address = soup.find_all (class_= 'list-card-addr')
        price = list(soup.find_all (class_='list-card-price'))
        beds = list(soup.find_all("ul", class_="list-card-details"))
        details = soup.find_all ('div', {'class': 'list-card-details'})
        home_type = soup.find_all ('div', {'class': 'list-card-footer'})
        last_updated = soup.find_all ('div', {'class': 'list-card-top'})
        brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
        link = soup.find_all (class_= 'list-card-link')


        #create dataframe columns out of variables
        df['prices'] = price
        df['address'] = address
        df['beds'] = beds


    #create empty url list
    urls = []

    #loop through url, pull the href and strip out the address tag
    for link in soup.find_all("article"):
        href = link.find('a',class_="list-card-link")
        if href == None:
            addresses = None
        else:
            addresses = href.find('address',class_="list-card-addr").get_text()
        urls.append(addresses)
        urls = [x for x in urls if x is not None]


    #import urls into a links column
    df['link'] = urls
    df['link'] = df['link'].astype('str')


    #remove html tags
    df['link'] = df['link'].replace(',', '', regex=True)
    df['link'] = df['link'].replace(' ', '-', regex=True)
    df['link'] = 'https://www.zillow.com/homedetails/' + df['link'].astype(str)

    #convert columns to str
    df['prices'] = df['prices'].astype('str')
    df['address'] = df['address'].astype('str')
    df['beds'] = df['beds'].astype('str')

    #remove html tags
    df['prices'] = df['prices'].replace('<div class="list-card-price">', ' ', regex=True)
    df['address'] = df['address'].replace('<address class="list-card-addr">', ' ', regex=True)
    df['prices'] = df['prices'].replace('</div>', ' ', regex=True)
    df['address'] = df['address'].replace('</address>', ' ', regex=True)
    df['prices'] = df['prices'].str.replace(r'\D', '')

    #remove html tags from beds column
    df['beds'] = df['beds'].replace('<ul class="list-card-details"><li>', ' ', regex=True)
    df['beds'] = df['beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li>', ' ', regex=True)
    df['beds'] = df['beds'].replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
    df['beds'] = df['beds'].replace('<!-- -->bds</abbr></li><li class="">', ' ', regex=True)
    df['beds'] = df['beds'].replace('<!-- -->ba</abbr></li><li class="">', ' ', regex=True)
    df['beds'] = df['beds'].replace('<!-- -->bd</abbr></li><li class="">', ' ', regex=True)
    df['beds'] = df['beds'].replace('<abbr class="list-card-label"> ', ' ', regex=True)
    df['beds'] = df['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li></ul>', ' ', regex=True)
    df['beds'] = df['beds'].replace('Studio</li><liclass=""', '0', regex=True)
    df['beds'] = df['beds'].replace('Studio</li><li>', '0', regex=True)
    df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Multi-family home for sale</li></ul>', 'Multi-family ', regex=True)
    df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- House for sale</li></ul>', 'House ', regex=True)
    df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Townhouse for sale</li></ul>', 'Townhouse ', regex=True)
    df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Multi-family home for sale</li></ul>', 'Multi-family ', regex=True)
    df['beds'] = df['beds'].replace(' <!-- -->sqft</abbr></li><li class="list-card-statusText">- House for sale</li></ul>', 'House ', regex=True)
    df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Condo for sale</li></ul>', 'Condo ', regex=True)
    df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Coming soon</li></ul>', 'Coming_soon ', regex=True)
    df['beds'] = df['beds'].replace(' <!-- -->sqft</abbr></li><li class="list-card-statusText">- New construction</li></ul>', ' New_construction ', regex=True)
    
    #split beds column into beds, bath and sq_feet
    df[['beds','baths','sq_feet', 'type']] = df.beds.str.split(expand=True)

    #remove commas from sq_feet and convert to float
    df.replace(',','', regex=True, inplace=True)

    #drop nulls
    df = df[(df['prices'] != '') & (df['prices']!= ' ')]

    #convert column to float
    df['prices'] = df['prices'].astype('float')
    # d['sq_feet'] = df['sq_feet'].astype('float')

    #remove spaces from link column
    df['link'] = df.link.str.replace(' ','')

    #rearrange the columns
    df = df[['prices', 'address', 'link', 'beds', 'baths', 'sq_feet', 'type']]

    try_link = []
    for link in soup.find_all('a'):
        a = link.get('href')
        try_link.append(a)


    df_2 = pd.DataFrame(try_link, columns = ['links'])
    df_2.dropna(inplace=True)
    df_2.drop_duplicates(inplace=True)
    try_link = df_2[df_2['links'].str.startswith('https://www.zillow.com/homedetails')]
    try_link.reset_index(inplace=True)


    df = df.merge(try_link, left_index=True, right_index=True)
    df = df.drop(['link','index'], axis =1)
    df_master = df_master.append(df, ignore_index = True) 

C:\Users\evan\AppData\Local\Temp\ipykernel_7280\3199858393.py:54: FutureWarning: The default value of regex will change from True to False in a future version.
  df['prices'] = df['prices'].str.replace(r'\D', '')
C:\Users\evan\AppData\Local\Temp\ipykernel_7280\3199858393.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master = df_master.append(df, ignore_index = True)
C:\Users\evan\AppData\Local\Temp\ipykernel_7280\3199858393.py:54: FutureWarning: The default value of regex will change from True to False in a future version.
  df['prices'] = df['prices'].str.replace(r'\D', '')
C:\Users\evan\AppData\Local\Temp\ipykernel_7280\3199858393.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master = df_master.append(df, ignore_index = True)
C:\Users\evan\AppData\Local\Temp\ipykernel_7280\3199858393.py:

In [4]:
df_master.drop_duplicates(inplace=True)
df_master.sort_values(by='prices', ascending=False, inplace=True)
#df_master['sq_feet'] = pd.to_numeric(df_master['sq_feet'], errors='coerce') 

#df_master['sq_feet'] =df_master['sq_feet'].apply(pd.to_numeric, errors='coerce')
df_master['sq_feet'] = df_master['sq_feet'].astype(float)
df_master['sqft_price'] = df_master.prices / df_master.sq_feet
df_master.to_csv('output.csv', index=False)

In [5]:
df_master

,prices,address,beds,baths,sq_feet,type,links,sqft_price
54,10000000.0,121 S Dexter Street Denver CO 80246,7,10,10268.0,House,https://www.zillow.com/homedetails/121-S-Dexte...,973.899494
74,3385000.0,90 S Ivy Street Denver CO 80224,6,6,6698.0,House,https://www.zillow.com/homedetails/1735-Centra...,505.374739
49,2985000.0,680 S Gilpin Street Denver CO 80209,5,5,6396.0,House,https://www.zillow.com/homedetails/680-S-Gilpi...,466.697936
18,2750000.0,568 S Gilpin Street Denver CO 80209,4,4,4442.0,House,https://www.zillow.com/homedetails/568-S-Gilpi...,619.090500
37,1895000.0,423 S Gilpin Street Denver CO 80209,4,4,3076.0,House,https://www.zillow.com/homedetails/423-S-Gilpi...,616.059818
51,1700000.0,770 N Lafayette Street Denver CO 80218,3,4,3236.0,House,https://www.zillow.com/homedetails/770-N-Lafay...,525.339926
87,1200000.0,29 S Oneida Street Denver CO 80230,4,4,3432.0,House,https://www.zillow.com/homedetails/29-S-Oneida...,349.650350
64,1150000.0,3056 Geneva Street Denver CO 80238,4,4,3606.0,House,https://www.zillow.com/homedetails/3056-Geneva...,318.912923
82,1149000.0,1420 Columbine Street Denver CO 80206,4,3,3339.0,House,https://www.zillow.com/homedetails/1420-Columb...,344.115004
40,985000.0,2560 Champa St Denver CO 80205,4,3,2297.0,House,https://www.zillow.com/homedetails/2560-Champa...,428.820200


In [6]:
# for i in soup3:
#         address = soup3.find_all (class_= 'list-card-addr')
#         price = list(soup3.find_all (class_='list-card-price'))
#         beds = list(soup3.find_all("ul", class_="list-card-details"))
#         details = soup3.find_all ('div', {'class': 'list-card-details'})
#         home_type = soup3.find_all ('div', {'class': 'list-card-footer'})
#         last_updated = soup3.find_all ('div', {'class': 'list-card-top'})
#         brokerage = list(soup3.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
#         link = soup3.find_all (class_= 'list-card-link')


# #create dataframe columns out of variables
# df['prices'] = price
# df['address'] = address
# df['beds'] = beds


# #create empty url list
# urls = []

# #loop through url, pull the href and strip out the address tag
# for link in soup3.find_all("article"):
#     href = link.find('a',class_="list-card-link")
#     if href == None:
#         addresses = None
#     else:
#         addresses = href.find('address',class_="list-card-addr").get_text()
#     urls.append(addresses)
#     urls = [x for x in urls if x is not None]


# #import urls into a links column
# df['link'] = urls
# df['link'] = df['link'].astype('str')


# #remove html tags
# df['link'] = df['link'].replace(',', '', regex=True)
# df['link'] = df['link'].replace(' ', '-', regex=True)
# df['link'] = 'https://www.zillow.com/homedetails/' + df['link'].astype(str)

# #convert columns to str
# df['prices'] = df['prices'].astype('str')
# df['address'] = df['address'].astype('str')
# df['beds'] = df['beds'].astype('str')

# #remove html tags
# df['prices'] = df['prices'].replace('<div class="list-card-price">', ' ', regex=True)
# df['address'] = df['address'].replace('<address class="list-card-addr">', ' ', regex=True)
# df['prices'] = df['prices'].replace('</div>', ' ', regex=True)
# df['address'] = df['address'].replace('</address>', ' ', regex=True)
# df['prices'] = df['prices'].str.replace(r'\D', '')

# #remove html tags from beds column
# df['beds'] = df['beds'].replace('<ul class="list-card-details"><li>', ' ', regex=True)
# df['beds'] = df['beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li>', ' ', regex=True)
# df['beds'] = df['beds'].replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
# df['beds'] = df['beds'].replace('<!-- -->bds</abbr></li><li class="">', ' ', regex=True)
# df['beds'] = df['beds'].replace('<!-- -->ba</abbr></li><li class="">', ' ', regex=True)
# df['beds'] = df['beds'].replace('<!-- -->bd</abbr></li><li class="">', ' ', regex=True)
# df['beds'] = df['beds'].replace('<abbr class="list-card-label"> ', ' ', regex=True)
# df['beds'] = df['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li></ul>', ' ', regex=True)
# df['beds'] = df['beds'].replace('Studio</li><liclass=""', '0', regex=True)
# df['beds'] = df['beds'].replace('Studio</li><li>', '0', regex=True)
# df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Multi-family home for sale</li></ul>', 'Multi-family ', regex=True)
# df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- House for sale</li></ul>', 'House ', regex=True)
# df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Townhouse for sale</li></ul>', 'Townhouse ', regex=True)
# df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Multi-family home for sale</li></ul>', 'Multi-family ', regex=True)
# df['beds'] = df['beds'].replace(' <!-- -->sqft</abbr></li><li class="list-card-statusText">- House for sale</li></ul>', 'House ', regex=True)
# df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Condo for sale</li></ul>', 'Condo ', regex=True)
# df['beds'] = df['beds'].replace('<!-- -->sqft</abbr></li><li class="list-card-statusText">- Coming soon</li></ul>', 'Coming_soon ', regex=True)

# #split beds column into beds, bath and sq_feet
# df[['beds','baths','sq_feet', 'type']] = df.beds.str.split(expand=True)

# #remove commas from sq_feet and convert to float
# df.replace(',','', regex=True, inplace=True)

# #drop nulls
# df = df[(df['prices'] != '') & (df['prices']!= ' ')]

# #convert column to float
# df['prices'] = df['prices'].astype('float')
# # d['sq_feet'] = df['sq_feet'].astype('float')

# #remove spaces from link column
# df['link'] = df.link.str.replace(' ','')

# #rearrange the columns
# df = df[['prices', 'address', 'link', 'beds', 'baths', 'sq_feet', 'type']]

# try_link = []
# for link in soup3.find_all('a'):
#     a = link.get('href')
#     try_link.append(a)


# df_2 = pd.DataFrame(try_link, columns = ['links'])
# df_2.dropna(inplace=True)
# df_2.drop_duplicates(inplace=True)
# try_link = df_2[df_2['links'].str.startswith('https://www.zillow.com/homedetails')]
# try_link.reset_index(inplace=True)


# df = df.merge(try_link, left_index=True, right_index=True)
# df = df.drop(['link','index'], axis =1)
# df